# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

#Importing sys to be able to see the key from different directories
import sys

sys.path.append("../")

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Import .csv file
city_weather_path = "cities.csv"
city_weather_data = pd.read_csv(city_weather_path)

# Preview dataframe
city_weather_data

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mahebourg,-44.179138,82.513939,78.80,61,40,11.41,MU,1602999113
1,1,carnarvon,-23.929630,94.861338,77.00,65,90,20.80,AU,1602999113
2,2,punta arenas,-85.063389,-103.115094,41.00,86,72,8.05,CL,1602999113
3,3,nizwa,22.076830,56.135380,88.34,9,16,1.03,OM,1602999114
4,5,viedma,-43.823507,-61.262479,75.60,30,0,14.52,AR,1602998818
...,...,...,...,...,...,...,...,...,...,...
652,706,dzaoudzi,-12.155872,46.358761,75.20,88,75,11.41,YT,1602999262
653,707,ilhabela,-25.038610,-44.447066,67.75,79,97,7.99,BR,1602999262
654,708,chilliwack,48.752202,-121.805235,50.29,89,100,2.21,CA,1602999004
655,709,boende,0.523678,21.113208,76.15,84,94,1.16,CD,1602999262


In [3]:
# Cleanup dataframe by dropping index column of csv file
city_weather_data.drop("Unnamed: 0", axis=1, inplace=True)  

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [5]:
# Store latitude and longitude in locations
locations = city_weather_data[["Lat", "Lng"]]

# Store humidity data in variable
humidity = city_weather_data["Humidity"]

In [6]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
max_humid = city_weather_data['Humidity'].max()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_humid,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Create a copy of dataframe and drop cities based on ideal weather conditions
ideal_weather_df = city_weather_data.copy()
ideal_weather_df.drop(ideal_weather_df[ideal_weather_df["Max Temp"] > 80].index, inplace = True)
ideal_weather_df.drop(ideal_weather_df[ideal_weather_df["Max Temp"] < 70].index, inplace = True)
ideal_weather_df.drop(ideal_weather_df[ideal_weather_df["Wind Speed"] > 10].index, inplace = True)
ideal_weather_df.drop(ideal_weather_df[ideal_weather_df["Cloudiness"] < 40].index, inplace = True)

# Preview dataframe
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,tabou,0.651787,-6.042790,75.99,91,63,6.35,CI,1602999114
23,lazaro cardenas,-1.317405,-112.131624,78.58,87,98,3.78,MX,1602999119
37,nauta,-3.964758,-74.670121,73.11,94,84,2.55,PE,1602999122
45,kahului,24.761240,-155.422879,78.15,66,40,3.36,US,1602998939
66,airai,6.098344,139.080619,77.11,51,75,2.77,TL,1602999130
...,...,...,...,...,...,...,...,...,...
623,ewa beach,12.557442,-161.254250,79.00,78,90,3.85,US,1602999079
642,kisangani,0.650071,25.046854,71.73,97,100,2.10,CD,1602999260
646,kibala,-10.789648,15.559109,73.40,88,75,2.24,TD,1602999261
648,yataity del norte,-24.853816,-56.255030,71.69,66,71,5.08,PY,1602999261


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Rename dataframe
hotel_df = ideal_weather_df

# Add Hotel Name column to dataframe
hotel_df["Hotel Name"] = ""

# Preview dataframe
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
5,tabou,0.651787,-6.042790,75.99,91,63,6.35,CI,1602999114,
23,lazaro cardenas,-1.317405,-112.131624,78.58,87,98,3.78,MX,1602999119,
37,nauta,-3.964758,-74.670121,73.11,94,84,2.55,PE,1602999122,
45,kahului,24.761240,-155.422879,78.15,66,40,3.36,US,1602998939,
66,airai,6.098344,139.080619,77.11,51,75,2.77,TL,1602999130,


In [9]:
# Params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location of each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_address = requests.get(base_url, params=params)
    
    # convert to json
    hotel_address = hotel_address.json()

    #The below code is for displaying the json file to understand what to reference
#     print(json.dumps(hotel_address, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_address["results"][0]["name"]
        print(f"Hotel found near {row['City']}. Populating data into dataframe.")
        
    except (KeyError, IndexError):
        print(f"No hotels found near {row['City']}... skipping.")

No hotels found near tabou... skipping.
No hotels found near lazaro cardenas... skipping.
No hotels found near nauta... skipping.
No hotels found near kahului... skipping.
No hotels found near airai... skipping.
No hotels found near kailua... skipping.
No hotels found near esperance... skipping.
Hotel found near certegui. Populating data into dataframe.
No hotels found near ipora... skipping.
No hotels found near avarua... skipping.
No hotels found near cabinda... skipping.
No hotels found near curup... skipping.
No hotels found near puerto ayora... skipping.
Hotel found near patancheru. Populating data into dataframe.
No hotels found near puerto escondido... skipping.
No hotels found near gunjur... skipping.
No hotels found near annigeri... skipping.
No hotels found near trat... skipping.
No hotels found near bonthe... skipping.
No hotels found near kapaa... skipping.
No hotels found near huilong... skipping.
No hotels found near coahuayana... skipping.
No hotels found near kabo... sk

In [10]:
# Preview dataframe
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
5,tabou,0.651787,-6.042790,75.99,91,63,6.35,CI,1602999114,
23,lazaro cardenas,-1.317405,-112.131624,78.58,87,98,3.78,MX,1602999119,
37,nauta,-3.964758,-74.670121,73.11,94,84,2.55,PE,1602999122,
45,kahului,24.761240,-155.422879,78.15,66,40,3.36,US,1602998939,
66,airai,6.098344,139.080619,77.11,51,75,2.77,TL,1602999130,
...,...,...,...,...,...,...,...,...,...,...
623,ewa beach,12.557442,-161.254250,79.00,78,90,3.85,US,1602999079,
642,kisangani,0.650071,25.046854,71.73,97,100,2.10,CD,1602999260,
646,kibala,-10.789648,15.559109,73.40,88,75,2.24,TD,1602999261,
648,yataity del norte,-24.853816,-56.255030,71.69,66,71,5.08,PY,1602999261,


In [11]:
# Drop cities that have no hotels near them
hotel_df = hotel_df[hotel_df["Hotel Name"].astype(bool)]

# Preview dataframe
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
110,certegui,5.306336,-76.743880,73.31,96,100,1.39,CO,1602999140,Puerto Salazar
136,patancheru,17.462555,78.260205,79.03,100,40,5.17,IN,1602999146,OYO 10188 Hotel DS Grand Inn
558,sungaipenuh,-1.827827,101.288582,79.77,59,81,5.46,ID,1602999242,Swarga Lodge and Homestay


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content = hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))